来源：https://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/

### 前言

现在两种流行的生成式模型：Generative Adversarial Nets (GAN)和Variational Autoencoder (VAE)。这两种模型有着本质上的不同，GAN基于博弈论，其目标是找到discriminator和generator间的纳什均衡；而VAE是基于贝叶斯推断，可以对数据的潜在分布建立概率分布模型，并从该分布中采集新样本。

### VAE的公式与直观含义

假设我们想生成某种数据，可以假设该种数据由某种隐变量（latent variable）生成。在生成式模型中首先确定隐变量是很有用的，没有隐变量，那么数据只能盲目地生成。这是GAN和VAE的一大区别。VAE使用了隐变量，所以模型有时会花费较大的代价。

可以定义如下记号：

$X$：我们想要生成的数据

$z$：隐变量

$P(X)$：数据$X$的分布

$P(z)$：隐变量$z$的分布

$P(X|z)$：在给定隐变量$z$时，$X$的分布

我们希望对数据建模，因此希望直到$P(X)$，其可以表示成如下形式：

$P(x)=\int P(X|z)P(z)dz$

在VAE中，可以利用$P(z|X)$来估计$P(z)$。这有着直观的含义：我们希望让隐变量与真实数据相关，即用那些最可能生成真实数据的隐变量的分布$P(z|X)$来估计$P(z)$。

如何估计$P(z|X)$？对分布的估计可以使用MCMC或者变分推断（Variational Inference，VI）。VI的核心思想利用一个简单分布（如高斯分布），不断逼近真实分布，即最小化两个分布间的KL散度。假设此处利用$Q(z|X)$来逼近$P(z|X)$，两者的KL散度表示如下：

$D_{KL}[Q(z|X)\parallel P(z|X)]=\sum_{z}Q(z|X)\log \frac{Q(z|X)}{P(z|x)} = E_{z\sim Q(z|X)}[\frac{Q(z|X)}{P(z|X)}] = E_{z\sim Q(z|X)}[\log Q(z|X) -\log P(z|X)]$

再由贝叶斯公式，上式可变成：

$E_{z\sim Q(z|X)}[\log Q(z|X) -\log P(z|X)] = E_{z\sim Q(z|X)}[\log Q(z|X) -\log \frac{P(X|z)P(z)}{P(X)}] = E_{z\sim Q(z|X)}[\log Q(z|X) -\log P(X|z)-\log P(z)+\log P(X)]$

由于$P(X)$与$z$无关，所以可以将其移到等式左边，于是有：

$D_{KL}[Q(z|X)\parallel P(z|X)]-\log P(X) = E_{z\sim Q(z|X)}[\log Q(z|X) -\log P(X|z)-\log P(z)]$

左右同时乘-1，有：$\log P(X) - D_{KL}[Q(z|X)\parallel P(z|X)] = E_{z\sim Q(z|X)}[\log P(X|z) - (\log Q(z|X)-\log P(z))] = E_{z\sim Q(z|X)}[\log P(X|z) - D_{KL}[Q(z|X)\parallel P(z)]]$

于是：

$\log P(X) - D_{KL}[Q(z|X)\parallel P(z|X)]= E_{z\sim Q(z|X)}[\log P(X|z) - D_{KL}[Q(z|X)\parallel P(z)]]$

上面的就是VAE的目标函数。$P(X)$可以看成一个定值，因此我们希望等式左右两边的值越大越好。左边的$P(X|z)$是给定隐变量时真实数据的分布，可以看作解码器（decoder）；$Q(z|X)$是编码器，$z$是隐变量。

### VAE的目标函数

想要最大化$E_{z\sim Q(z|X)}[\log P(X|z) - D_{KL}[Q(z|X)\parallel P(z)]]$，就要最大化$E_{z\sim Q(z|X)}[P(X|z)]$，最小化$E_{z\sim Q(z|X)}[D_{KL}[Q(z|X)\parallel P(z)]]$。

前个目标可以通过极大似然估计解决，将$z$看成输入，$X$看成输出，那么$P(X|z)$就是似然概率。通过极大似然估计就能得到该分布的参数的较好的估计。

关于$D_{KL}[Q(z|X)\parallel P(z)]$，这里$P(z)$是隐变量的分布，我们在生成样本时要对$z$进行采样，所以这里可以将$z$设为服从较易采样的高斯分布$N(0,1)$。于是现在的目标就变成了让$Q(z|X)$尽可能接近$N(0,1)$。

假设我们希望$Q(z|X)$服从$N(\mu(X),\Sigma(X))$，此时KL散度有闭式解：

$D_{KL}[N(\mu(X),\Sigma(X))\parallel N(0,1)]=\frac{1}{2}(tr(\Sigma(X))+\mu(X)^T \mu(X)-k-\log det(\Sigma(X)))$

可以将$\Sigma(X)$实现为一个向量，于是上式可以化简为：

$D_{KL}[N(\mu(X),\Sigma(X))\parallel N(0,1)]=\frac{1}{2}\sum_{k}(\Sigma(X)+\mu(X)^2 -1-\log \Sigma(X))$

在实践中，可将$\log\Sigma(X)$建模为$\Sigma(X)$。因为计算指数比计算对数更数值稳定，于是有：

$D_{KL}[N(\mu(X),\Sigma(X))\parallel N(0,1)]=\frac{1}{2}\sum_{k}(\exp(\Sigma(X))+\mu(X)^2 -1-\Sigma(X))$

### VAE的实现